In [153]:
from keras.applications import VGG16

In [154]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

#Loads the VGG16 model 
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

In [173]:
#model.layers

In [156]:
model.layers[1].trainable

True

In [157]:
for i in model.layers:
    i.trainable = False

In [158]:
model.layers[1].trainable

False

In [159]:
from keras.models import Sequential
from keras.layers import Dense , Flatten
from keras.models import Model

In [160]:
model.output

<tf.Tensor 'block5_pool_10/MaxPool:0' shape=(None, 7, 7, 512) dtype=float32>

In [161]:
top_model = model.output

In [162]:
top_model = Flatten(name = "flatten")(top_model)
top_model = Dense(256,activation='relu')(top_model)
top_model = Dense(2, activation = "sigmoid")(top_model)

In [163]:
#compilig the new model
newmodel = Model(inputs=model.input,outputs=top_model)

In [174]:
#newmodel.layers

In [165]:
newmodel.output

<tf.Tensor 'dense_14/Sigmoid:0' shape=(None, 2) dtype=float32>

In [169]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = './dataset/train/'
validation_data_dir = './dataset/test/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 2565 images belonging to 2 classes.
Found 635 images belonging to 2 classes.


In [170]:
train_generator.class_indices

{'jyotirmaya': 0, 'niilam': 1}

In [171]:
validation_generator.class_indices

{'jyotirmaya': 0, 'niilam': 1}

In [172]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("nface_classifier.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
newmodel.compile(loss = 'binary_crossentropy',
              optimizer = Adam(),
              metrics = ['accuracy'])

nb_train_samples = 2565
nb_validation_samples = 635
epochs = 3
batch_size = 16

history = newmodel.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

newmodel.save("nface_classifier.h5")

Epoch 1/3
160/160 [==============================] - 474s 3s/step - loss: 0.1390 - accuracy: 0.9735 - val_loss: 1.0190e-04 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.00010, saving model to nface_classifier.h5
Epoch 2/3
160/160 [==============================] - 467s 3s/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 1.2718e-05 - val_accuracy: 1.0000

Epoch 00002: val_loss improved from 0.00010 to 0.00001, saving model to nface_classifier.h5
Epoch 3/3
160/160 [==============================] - 522s 3s/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 1.2815e-07 - val_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.00001 to 0.00000, saving model to nface_classifier.h5
